In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm, tqdm_notebook

from scipy import stats
from sklearn.model_selection import GroupKFold

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.utils.data
from transformers import *

import os
import re
import math
import random
from matplotlib import pyplot as plt
import warnings
from math import floor, ceil

warnings.filterwarnings('ignore')
device = torch.device('cuda')
#device = torch.device('cpu')
torch.backends.cudnn.benchmark=True

%matplotlib inline

c:\Users\Lab000\anaconda3\envs\contrastive_learning2\lib\site-packages\transformers\generation_utils.py:27: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  FutureWarning,
c:\Users\Lab000\anaconda3\envs\contrastive_learning2\lib\site-packages\transformers\generation_tf_utils.py:27: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  FutureWarning,
c:\Users\Lab000\anaconda3\envs\contrastive_learning2\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
output_dir="./token_model_config/"

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

def spearman_corr(y_true, y_pred):
    if np.ndim(y_pred) == 2:
        corr = np.nan_to_num([stats.spearmanr(y_true[:, i], y_pred[:, i])[0] for i in range(y_true.shape[1])]).mean()
    else:
        corr = stats.spearmanr(y_true, y_pred)[0]
    return corr
  
def calc_each_spearman(valid_y, valid_pred):
    lst = []
    for idx in range(30):
        spearman = spearman_corr(valid_y[:,idx], valid_pred[:,idx])
        lst.append(spearman)
    df = pd.DataFrame(lst).T
    df.columns = class_names
    return df

In [4]:
def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    first_sep = True
    current_segment_id = 0
    
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input(title, question, answer, max_sequence_length=512-1, 
                t_max_len=70-1, q_max_len=219, a_max_len=219):#???

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))

        t = t[:t_new_len]
        q = norm_token_length(q, q_new_len)
        a = norm_token_length(a, a_new_len)
    
    return t, q, a

def norm_token_length(tokens, l):
    if len(tokens) > l:
        head = l//2
        tail = l - head
        return tokens[:head] + tokens[-tail:]
    else:
        return tokens[:l]

def _convert_to_bert_inputs(title, question, answer, cate, max_sequence_length=512):
    """Converts tokenized input to ids, masks and segments for BERT"""
    stoken = ["[CLS]"] + [cate] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)
    
    return [input_ids, input_segments]

def convert_row(row):
    #c = f"[{row['category'].lower()}]"
    c = f"[{row['category']}]"
    t, q, a = row["question_title"], row["question_body"], row["answer"]
    t, q, a = _trim_input(t, q, a)
    ids, segments = _convert_to_bert_inputs(t, q, a, c)
    
    return np.array([[ids, segments]])

In [5]:


train = pd.read_csv('../input/google-quest-challenge/train.csv').fillna(' ')
sub = pd.read_csv('../input/google-quest-challenge/sample_submission.csv').fillna(' ')

#model_class, tokenizer_class = transformer_models_dict[pretrained_weights]
#tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

categories = train["category"].unique().tolist()
categories = [f"[{c}]" for c in categories]
#tokenizer.add_tokens(categories)#??

#tokenizer.added_tokens_encoder#??

In [6]:
model="bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model)
tokenizer.add_tokens(categories)


loading configuration file config.json from cache at C:\Users\Lab000/.cache\huggingface\hub\models--bert-base-cased\snapshots\5532cc56f74641d4bb33641f5c76a55d11f846e0\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at C:\Users\Lab000/.cache\huggingface\hub\models--bert-base-cased\snapshots\5532cc56f74641d4bb33641f5c76a55d11f846e0\vocab.txt
l

5

In [7]:
out_path="./bert_based_tokenizer"
tokenizer.save_pretrained(output_dir+"tokenizer")

tokenizer config file saved in ./token_model_config/tokenizer\tokenizer_config.json
Special tokens file saved in ./token_model_config/tokenizer\special_tokens_map.json


('./token_model_config/tokenizer\\tokenizer_config.json',
 './token_model_config/tokenizer\\special_tokens_map.json',
 './token_model_config/tokenizer\\vocab.txt',
 './token_model_config/tokenizer\\added_tokens.json',
 './token_model_config/tokenizer\\tokenizer.json')

In [8]:
%%time
X = train.apply(convert_row, axis=1).values
X = np.vstack(X).reshape((len(X), 1024))
assert X.shape == (6079, 1024)

Token indices sequence length is longer than the specified maximum sequence length for this model (649 > 512). Running this sequence through the model will result in indexing errors


Wall time: 6.91 s


In [9]:
train.loc[0]["question_title"]

'What am I losing when using extension tubes instead of a macro lens?'

In [10]:
X[0][:512]

array([  101, 28996,  1327,  1821,   146,  3196,  1165,  1606,  4973,
       11182,  1939,  1104,   170, 23639,  2180, 11039,   136,   102,
        1258,  1773,  1213,  1114, 23639,  2180,  6427,  1113,   118,
        1103,   118, 10928,   113,  2373,   131, 11802, 11039,   117,
        1231,  1964,   119, 11039,  5378,  1113,   170,  2632, 11039,
         117, 14403,  4973, 11182,   114,   117,   146,  1156,  1176,
        1106,  1243,  1748,  1114,  1142,   119,  1109,  2645,  1114,
        1103,  4884,   146,  1215,  1110,  1115,  2817,  1110,  9506,
        1105, 22769,  1654,  1110, 20405,  1120,  1436,   119,  1188,
        2609,  1139, 18011,  1106,  1253,  5174,   113,  2373,   131,
        2044,  9895,   114,  1986,   117,  1112,  3450,  1110,  8320,
         117,   146,  1328,  1106,  1129,  1682,  1106,  5211,  1686,
        9895,   119,   146,  2059,  1115,  1111,  1142,   117, 12365,
       14467,  6697,  1105,  1383,  8637, 22769,  1209,  1129,  1104,
        1632,  1494,

In [11]:
X[0][512:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [12]:
class_names = list(sub.columns[1:])
y = train[class_names].values#(6079,30)

lst = []
for idx in range(30):
    t = pd.DataFrame(y[:,idx])[0]
    # print(len(t))
    #print(1-t.value_counts())
    #print(1-t.value_counts()/len(t))
    w_df = (1-t.value_counts()/len(t)).reset_index()
    #print(w_df)
    w_dic = {row["index"]: row[0] for _, row in w_df.iterrows()}
    # print("=======================")
    # print(w_dic)
    w = t.map(w_dic).values#(6079,)
    #print(w.shape)
    lst.append(w)
# print(lst)
# print("==================================")
weights = np.vstack(lst).T#轉置, shape:(6079,30)

import copy
y_true = copy.deepcopy(y)
y = np.hstack([y, weights])#(6079,60)


In [13]:
X[:, 512:].min()

0

In [14]:
def custom_loss(data, targets):
    
    mse = nn.MSELoss(reduction="none")(data[:,:30].sigmoid(), targets[:,:30])
    bce = nn.BCEWithLogitsLoss(reduction='none')(data[:,:30], targets[:,:30])#??
    w =  targets[:,30:]
    #loss = (mse*w).sum() + bce.sum()
    loss = (mse).sum()+ bce.sum()
    return loss

class CustomBert(nn.Module):
    def __init__(self, model,config_path=None):
        super(CustomBert, self).__init__()
        self.config = AutoConfig.from_pretrained(model) 
        
        self.config.num_labels = 30
        self.config.output_hidden_states = True
        self.n_use_layer = 4
        self.n_labels = self.config.num_labels
        #self.config.save_pretrained("bert_based_config")
        self.config.save_pretrained(output_dir+"config")
        #self.bert = BertModel(config)
        self.bert=AutoModel.from_pretrained(model, config=self.config)
        #self.bert.save_pretrained('bert_based_model')
        self.bert.save_pretrained(output_dir+"model")
        self.dense1 = nn.Linear(self.config.hidden_size*self.n_use_layer, self.config.hidden_size*self.n_use_layer)
        self.dense2 = nn.Linear(self.config.hidden_size*self.n_use_layer, self.config.hidden_size*self.n_use_layer)
        self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
        self.classifier = nn.Linear(self.config.hidden_size*self.n_use_layer, self.config.num_labels)
        #self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,
                position_ids=None, head_mask=None, inputs_embeds=None, labels=None):

        # outputs = self.bert(input_ids,
        #                     attention_mask=attention_mask,
        #                     token_type_ids=token_type_ids,
        #                     position_ids=position_ids,
        #                     head_mask=head_mask,
        #                     inputs_embeds=inputs_embeds)
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids
                            )
        
        #print(outputs[2][-1].shape)
        pooled_output = torch.cat([outputs[2][-1*(i+1)][:,0] for i in range(self.n_use_layer)], dim=1)#把倒數最後4個layer的cls output concat在一起，把4個(8,768) concat，變成(8,3072)
        
        pooled_output = self.dense1(pooled_output)
        pooled_output = self.dense2(pooled_output)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]
        
        return outputs

In [17]:
model=CustomBert(model)

loading configuration file config.json from cache at C:\Users\Lab000/.cache\huggingface\hub\models--bert-base-cased\snapshots\5532cc56f74641d4bb33641f5c76a55d11f846e0\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

Configuration saved in ./token_model_config/config\config.json
loading weights file pytorch_model.bin from cache at C:\Users\Lab000/.cache\huggingface\hub\models-

In [19]:
param_optimizer = list(model.named_parameters())
#print(param_optimizer[0])
print("==============================")
print(param_optimizer[0][0])
print("==============================")
print(param_optimizer[0][1].shape)

bert.embeddings.word_embeddings.weight
torch.Size([28996, 768])


In [29]:
learning_rate = 5e-5
# optimizer setting
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.weight', 'LayerNorm.bias']
optimizer_grouped_parameters = []
max_lrs = []
for param in param_optimizer:
    if any(n in param[0] for n in no_decay):#weight_decay
      weight_decay = 0.0
    else:
      weight_decay = 0.1
    if param[0].find("bert.encoder.layer") != -1:
      
      n_diff_last = 11 - int(param[0].split(".")[3])
      #print(param[0].split("."))
      #print(learning_rate*0.9)
      lr = learning_rate*0.9**n_diff_last#**:平方,越底層encoder layer,lr越小
      #print(lr)
    elif "embeddings" in param[0]:
      
      lr = learning_rate*0.9**11
    else:
      lr = learning_rate
    max_lrs.append(lr)
    d = {"params": param[1], "weight_decay": weight_decay}
    optimizer_grouped_parameters.append(d)

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias


In [35]:
print(len(param_optimizer))
print(len(max_lrs))
print(len(optimizer_grouped_parameters))

205
205
205


In [17]:

N_FOLD=10
N_BERT_LABEL = 30
SEED = 42
BS = 8
# parameter
n_epoch = 3
learning_rate = 5e-5
max_grad_norm = 1.0

gkf = GroupKFold(n_splits=N_FOLD).split(X=train["question_body"], groups=train["question_body"])#??

spearman_scores = []
best_spearman_lst = []
losses_lst = []
epoch_spearman_lst = []
lr_lst_lst = []
each_speaman_dfs = []
#model=CustomBert(model)
#torch.save(model.config, 'config.pth')
for fold, (train_idx, valid_idx) in enumerate(gkf):#??
  # print(train_idx)
  # print("=======================")
  # print(valid_idx)
  # if fold in [0,1,2,3,4,5,6,7]:
  #   continue

  seed_everything(SEED)

  # Load Model
#   config = BertConfig.from_pretrained(pretrained_weights)
#   model = CustomBert.from_pretrained(pretrained_weights, config=config)
  model=CustomBert(model)
  model = model.to(device)
  model.bert.resize_token_embeddings(len(tokenizer))#??
  model = model.train()
  
  # optimizer setting
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'LayerNorm.weight', 'LayerNorm.bias']
  optimizer_grouped_parameters = []
  max_lrs = []
  for param in param_optimizer:
    if any(n in param[0] for n in no_decay):#weight_decay
      weight_decay = 0.0
    else:
      weight_decay = 0.1
    if param[0].find("bert.encoder.layer") != -1:
      
      n_diff_last = 11 - int(param[0].split(".")[3])
      lr = learning_rate*0.9**n_diff_last
    elif "embeddings" in param[0]:
      lr = learning_rate*0.9**11
    else:
      lr = learning_rate
    max_lrs.append(lr)
    d = {"params": param[1], "weight_decay": weight_decay}
    optimizer_grouped_parameters.append(d)
  optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, correct_bias=True)

  # print(train_idx)
  # print("===========================")
  # print(valid_idx)
  
  # train valid split
  train_x = X[train_idx]
  valid_x = X[valid_idx]
  train_y = y[train_idx]
  valid_y = y[valid_idx]
  
  # set loader  
  train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_x, dtype=torch.long), torch.tensor(train_y, dtype=torch.float))
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BS, shuffle=True)
  valid_dataset = torch.utils.data.TensorDataset(torch.tensor(valid_x, dtype=torch.long), torch.tensor(valid_y, dtype=torch.float))
  valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BS, shuffle=False)

  # set schedueler
  num_training_steps = len(train_loader)*n_epoch
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lrs, total_steps=num_training_steps)

  model.zero_grad()
  optimizer.zero_grad()
   
  best_spearman = 0
  losses = []
  epoch_spearman = []
  lr_lst = []
  for epoch in range(n_epoch):
    lr = np.array([param_group["lr"] for param_group in optimizer.param_groups]).mean()
    tk0 = tqdm_notebook(enumerate(train_loader), total=len(train_loader), leave=False)
    for i, (x_batch, y_batch) in tk0:
      input_ids = x_batch[:, :512]
      token_ids = x_batch[:, 512:]
      #print((input_ids > 0))
      #print(token_ids.max())
      #mask=(input_ids > 0).type(torch.uint8)
      #print(mask) 
      y_pred = model(input_ids.to(device), attention_mask=(input_ids > 0).to(device), token_type_ids=token_ids.to(device))
      #y_pred = model(input_ids.to(device), attention_mask=mask.to(device), token_type_ids=token_ids.to(device))
      loss = custom_loss(y_pred[0], y_batch.to(device))
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) 
      optimizer.step()#更新模型
      optimizer.zero_grad()
      scheduler.step()#更新學習率
      lr_lst.append(np.array([param_group["lr"] for param_group in optimizer.param_groups]).mean())#每個layer的lr取平均
      losses.append(float(loss))

    # epoch validation
    for param in model.parameters():
      param.requires_grad=False
    model.eval()

    lst = []
    sum_loss = 0
    for i, (x_batch, y_batch)  in enumerate(valid_loader):
      input_ids = x_batch[:, :512]
      token_ids = x_batch[:, 512:]
      
      with torch.no_grad():
        y_pred = model(input_ids.to(device), attention_mask=(input_ids > 0).to(device), token_type_ids=token_ids.to(device))
        loss = custom_loss(y_pred[0], y_batch.to(device))

      lst.append(y_pred[0].sigmoid().cpu().squeeze().numpy())
      sum_loss += loss.cpu().squeeze().numpy()
    valid_pred = np.vstack(lst)#(608,30)
    
    ave_loss = sum_loss/len(valid_loader)

    spearman_score = spearman_corr(valid_y[:,:N_BERT_LABEL], valid_pred)  
    epoch_spearman.append(spearman_score)
    
    for param in model.parameters():
      param.requires_grad=True
    model.train()
    model_name="bert-based-case"
    # print(f"{model}_f{fold}_best")
    # print("=======================================")
    if best_spearman <= spearman_score:
      #torch.save(model.state_dict(), f"{model_name}_f{fold}_best")
      torch.save(model.state_dict(), f"./bert_based_case/bce_no_opt_binning/{model_name}_f{fold}_best")
      best_spearman = spearman_score
      each_speaman_df = calc_each_spearman(valid_y[:,:N_BERT_LABEL], valid_pred)
      display(each_speaman_df)

    print(f"fold-{fold} epoch {epoch}: {spearman_score} / loss avg: {ave_loss}")
    
  best_spearman_lst.append(best_spearman)
  losses_lst.append(losses)
  epoch_spearman_lst.append(epoch_spearman)
  lr_lst_lst.append(lr_lst)
  each_speaman_dfs.append(each_speaman_df)

  torch.cuda.empty_cache()

  0%|          | 0/684 [00:00<?, ?it/s]

(tensor([[ 4.4077e-01,  1.0898e-01, -1.3905e-01,  1.3382e-01, -1.8977e-01,
          5.8466e-02, -6.0865e-02, -4.6442e-02, -7.9957e-02,  6.7435e-02,
          2.8622e-01,  1.0170e-01, -5.3567e-02,  2.6041e-02,  8.5417e-02,
         -2.6987e-01,  6.2995e-02, -8.0611e-02, -1.5625e-01, -3.5013e-02,
         -2.4774e-01,  5.1169e-02,  2.6357e-01, -2.8071e-01,  1.8681e-01,
         -6.9078e-02,  6.0436e-02,  1.0242e-01,  5.0795e-02,  3.6374e-01],
        [ 3.7770e-01,  2.8610e-01, -2.8025e-01, -1.4926e-02,  3.7736e-02,
         -8.6002e-02, -7.0509e-02,  3.8240e-02,  5.2002e-02,  2.1216e-01,
          1.5438e-01,  1.3735e-01, -1.4858e-01,  2.3128e-02,  7.9482e-02,
         -1.1651e-01, -2.7527e-02, -8.4087e-02, -9.9185e-02, -1.0387e-01,
         -1.3924e-01,  1.5639e-01,  3.1927e-01, -3.4362e-01,  1.9307e-01,
         -1.5730e-01,  6.2441e-02,  1.1425e-01, -4.6488e-02,  4.8786e-01],
        [ 3.3686e-01,  2.5519e-01, -1.3893e-01,  2.8252e-01, -8.2711e-02,
          3.0985e-02, -1.2437e-01, 

KeyboardInterrupt: 

In [ ]:
len(tokenizer)

29001